In [1]:
# from circuit_breaking.src import *
%load_ext autoreload
%autoreload 2
import torch
from functools import partial
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm.auto import tqdm
print(os.getcwd())
import sys
from transformers import AutoTokenizer, AutoModelForCausalLM
import contextlib
import einops
import wandb
import pandas as pd
import dotenv
import bitsandbytes as bnb
from collections import defaultdict
dotenv.load_dotenv()

from huggingface_hub import login

HF_ACCESS_TOKEN = os.getenv("HF_ACCESS_TOKEN")
WANDB_API_KEY = os.getenv("WANDB_API_KEY")
if HF_ACCESS_TOKEN:
    login(token=HF_ACCESS_TOKEN)
    print("Successfully authenticated with Hugging Face.")
else:
    print("Hugging Face access token not found in environment variables.")

if WANDB_API_KEY:
    os.environ["WANDB_API_KEY"] = WANDB_API_KEY
    print("Successfully authenticated with Weights & Biases.")
else:
    print("Weights & Biases API key not found in environment variables.")


/root/sae-editing
Successfully authenticated with Hugging Face.
Successfully authenticated with Weights & Biases.


In [2]:
model_name_or_path = "google/gemma-2-9b"
model_type = "gemma-2"
other_model_type = "gemma2_9b"
# pretrained_path = "/data/huggingface/models--google--gemma-2-9b/snapshots/33c193028431c2fde6c6e51f29e6f17b60cbfac6/"
# pretrained_path = "/data/huggingface/models--google--gemma-2-9b-it/snapshots/11c9b309abf73637e4b6f9a3fa1e92e615547819/"
# pretrained_path = "gemma-2-rmu-fullrank"
pretrained_path = "gemma-2-rmu-fullrank"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

left_tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
left_tokenizer.pad_token_id = left_tokenizer.eos_token_id
left_tokenizer.padding_side = "left"

dtype = torch.bfloat16
if pretrained_path is not None:
    model = AutoModelForCausalLM.from_pretrained(pretrained_path, torch_dtype=dtype)
else:
    model = AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_dtype=dtype)
model.cuda()
n_layers = 42
n_heads = 16
n_kv_heads = 8

param_count_dict = {"attn.hook_q": 3584*4096, "attn.hook_k": 3584*2048, "attn.hook_v": 3584*2048, "attn.hook_result": 4096*3584, "mlp.hook_pre": 3584 * 14336, "mlp.hook_post": 14336 * 3584, "mlp.hook_gate": 3584 * 14336}
mmlu_batch_size = 2

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [6]:
from tasks.wmdp.WMDP_MCTask import WMDP_MCTask, WMDP_DedupedTask
from tasks.wmdp.WMDP_RelearnTask import WMDP_RelearnTask
from tasks.general_capabilities.MCTask_redo import run_general_evals
batch_size = 8
bio_mc_task = WMDP_MCTask(batch_size=batch_size, tokenizer=tokenizer, subset="wmdp-bio", shuffle=True)
num_iters = len(bio_mc_task.dataset) // batch_size

Map:   0%|          | 0/1273 [00:00<?, ? examples/s]

In [15]:
bio_mc_deduped_task = WMDP_DedupedTask(batch_size=batch_size, tokenizer=tokenizer, subset="wmdp-bio", shuffle=True)
num_iters_deduped = len(bio_mc_deduped_task.train_dataset) // batch_size

bio_mc_deduped_task.get_test_accuracy(model, use_test_data=False, num_iters=num_iters_deduped)
print(bio_mc_deduped_task.get_test_loss(model, n_iters=5))

last_logits.argmax(dim=1)=tensor([608, 599, 585, 585, 585, 599, 608, 608], device='cuda:0'), tokenized_labels=tensor([586, 599, 586, 585, 585, 599, 608, 586], device='cuda:0'), probs[range(len(last_logits)), tokenized_labels]=tensor([0.1523, 0.6406, 0.1709, 0.7656, 0.4414, 0.8359, 0.8203, 0.1143],
       device='cuda:0', dtype=torch.bfloat16)
last_logits.argmax(dim=1)=tensor([586, 586, 599, 608, 608, 599, 599, 599], device='cuda:0'), tokenized_labels=tensor([585, 586, 586, 608, 608, 599, 599, 599], device='cuda:0'), probs[range(len(last_logits)), tokenized_labels]=tensor([0.2168, 0.8828, 0.3203, 0.8164, 0.6836, 0.6836, 0.6250, 0.6992],
       device='cuda:0', dtype=torch.bfloat16)
last_logits.argmax(dim=1)=tensor([586, 586, 585, 599, 586, 585, 586, 585], device='cuda:0'), tokenized_labels=tensor([586, 586, 585, 585, 585, 586, 586, 585], device='cuda:0'), probs[range(len(last_logits)), tokenized_labels]=tensor([0.4668, 0.6992, 0.5508, 0.0869, 0.0386, 0.1416, 0.2656, 0.6562],
       devi

In [43]:
# from datasets import load_dataset
# original_bio_df = load_dataset("cais/wmdp", "wmdp-bio", split="test").to_pandas()
# original_cyber_df = load_dataset("cais/wmdp", "wmdp-cyber", split="test").to_pandas()

# # load in deduped data, split into train and test
# bio_train_dfs = []
# cyber_train_dfs = []
# for train_split_idx in [0, 1, 2, 3]:
#     # tasks/wmdp/data/mcq_split_0.jsonl
#     mcq_formatted_data = pd.read_json(f"tasks/wmdp/data/mcq_split_{train_split_idx}.jsonl", lines=True)
#     bio_indices = mcq_formatted_data["question"].isin(original_bio_df["question"])
#     cyber_indices = mcq_formatted_data["question"].isin(original_cyber_df["question"])
#     # display(bio_indices + cyber_indices)
#     # display(mcq_formatted_data[~(bio_indices + cyber_indices)])
#     print("Bio dataset is size ", len(mcq_formatted_data[bio_indices]), "and cyber dataset is size ", len(mcq_formatted_data[cyber_indices]), "Missed ", len(mcq_formatted_data[~(bio_indices + cyber_indices)]), "data points in train split ", train_split_idx)
#     bio_train_dfs.append(mcq_formatted_data[bio_indices])
#     cyber_train_dfs.append(mcq_formatted_data[cyber_indices])

# bio_test_dfs = []
# cyber_test_dfs = []
# for test_split_idx in [4]:
#     mcq_formatted_data = pd.read_json(f"tasks/wmdp/data/mcq_split_{test_split_idx}.jsonl", lines=True)
#     bio_indices = mcq_formatted_data["question"].isin(original_bio_df["question"])
#     cyber_indices = mcq_formatted_data["question"].isin(original_cyber_df["question"])
#     bio_test_dfs.append(mcq_formatted_data[bio_indices])
#     cyber_test_dfs.append(mcq_formatted_data[cyber_indices])
#     print("Bio dataset is size ", len(mcq_formatted_data[bio_indices]), "and cyber dataset is size ", len(mcq_formatted_data[cyber_indices]), "Missed ", len(mcq_formatted_data[~(bio_indices + cyber_indices)]), "data points in test split ", test_split_idx)

# bio_train_df = pd.concat(bio_train_dfs, ignore_index=True)
# cyber_train_df = pd.concat(cyber_train_dfs, ignore_index=True)
# bio_test_df = pd.concat(bio_test_dfs, ignore_index=True)
# cyber_test_df = pd.concat(cyber_test_dfs, ignore_index=True)
# from datasets import Dataset, DatasetDict
# # convert to huggingface dataset
# bio_train_dataset = Dataset.from_pandas(bio_train_df)
# cyber_train_dataset = Dataset.from_pandas(cyber_train_df)
# bio_test_dataset = Dataset.from_pandas(bio_test_df)
# cyber_test_dataset = Dataset.from_pandas(cyber_test_df)

# # Create separate DatasetDicts for bio and cyber
# bio_dataset = DatasetDict({
#     "train": bio_train_dataset,
#     "test": bio_test_dataset
# })

# cyber_dataset = DatasetDict({
#     "train": cyber_train_dataset,
#     "test": cyber_test_dataset
# })
# bio_dataset.push_to_hub("PhillipGuo/wmdp-deduped", "wmdp-bio")
# cyber_dataset.push_to_hub("PhillipGuo/wmdp-deduped", "wmdp-cyber")

Bio dataset is size  71 and cyber dataset is size  79 Missed  7 data points in train split  0
Bio dataset is size  71 and cyber dataset is size  76 Missed  10 data points in train split  1
Bio dataset is size  71 and cyber dataset is size  84 Missed  2 data points in train split  2
Bio dataset is size  71 and cyber dataset is size  79 Missed  7 data points in train split  3
Bio dataset is size  71 and cyber dataset is size  75 Missed  11 data points in test split  4


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/508 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/PhillipGuo/wmdp-deduped/commit/6247bd97fef16a2024b12c874b2badaa49bae202', commit_message='Upload dataset', commit_description='', oid='6247bd97fef16a2024b12c874b2badaa49bae202', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/PhillipGuo/wmdp-deduped', endpoint='https://huggingface.co', repo_type='dataset', repo_id='PhillipGuo/wmdp-deduped'), pr_revision=None, pr_num=None)

In [ ]:
def eval_callback(model, epoch, evaluate_every=10):
    if (epoch+1) % evaluate_every == 0:
        mmlu_score = run_general_evals(model, model_type=model_type, evals_to_include=["MMLU"], verbose=False, batch_size=2, device="cuda")["MMLU"]
        bio_acc = bio_mc_task.get_test_accuracy(model, num_iters=num_iters)
        return {"MMLU": mmlu_score, "bio_acc": bio_acc}
    else:
        return {}
    
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
def do_relearning(model, train_tasks, n_iters, grad_accum_steps=1, finetune_lora=False, lora_kwargs={'rank': 256, 'alpha': 32, 'dropout': 0.05, 'target_modules': 'all-linear'}, learning_kwargs={'lr': 1e-5, 'weight_decay': 0, 'use_cosine': False}, eval_callback_fn=None, forget_kwargs=None, maintain_kwargs=None, inject_label=None):
    # can either finetune full or lora

    if not finetune_lora:
        optimizer = bnb.optim.AdamW8bit(model.parameters(), lr=learning_kwargs['lr'], weight_decay=learning_kwargs['weight_decay'])

    elif finetune_lora:
        peft_config = LoraConfig(
            inference_mode=False,
            r=lora_kwargs['rank'],
            lora_alpha=lora_kwargs['alpha'],
            lora_dropout=lora_kwargs['dropout'],
            target_modules = lora_kwargs['target_modules'], #["q_proj", "v_proj", 
        )

        model = get_peft_model(model, peft_config).cuda()
        # model.print_trainable_parameters()
        print(f"Parameters in peft: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_kwargs['lr'], weight_decay=learning_kwargs['weight_decay'])
    
    if learning_kwargs['use_cosine']:
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=n_iters)

    train_losses = defaultdict(list)
    test_losses = []

    for iter_idx in tqdm(range(n_iters)):
        optimizer.zero_grad()
        for task_name, (task, task_weight) in train_tasks.items():
            task_loss = 0
            for i in range(grad_accum_steps):
                loss = task.get_train_loss(model) / grad_accum_steps
                task_loss += loss.item()
                # print(loss.item())
                (loss * task_weight).backward()
            train_losses[task_name].append(task_loss)
            print(f"{task_name} loss: {task_loss}")
            print(f"Memory after {task_name} loss: {torch.cuda.memory_allocated() / 10**9} GB")

        optimizer.step()
        optimizer.zero_grad()
        if learning_kwargs['use_cosine']:
            scheduler.step()
        torch.cuda.empty_cache()
        print(f"Memory after optimizer step: {torch.cuda.memory_allocated() / 10**9} GB")

        if eval_callback_fn is not None:
            test_losses.append(eval_callback_fn(model, epoch=iter_idx, forget_kwargs=forget_kwargs, maintain_kwargs=maintain_kwargs, inject_label=inject_label))
            print(test_losses[-1])

    if len(test_losses) > 0:
        return train_losses, test_losses
    return train_losses


# del model

# for name, model, mask, regular_evals, side_effect_evals, adversarial_evals in [("localized", localized_model, localized_mask, localized_regular_evals, localized_side_effect_evals, localized_adversarial_evals), ("nonlocalized", nonlocalized_model, nonlocalized_mask, nonlocalized_regular_evals, nonlocalized_side_effect_evals, nonlocalized_adversarial_evals)]:

relearning_regular_results = {}
# relearning_adversarial_results = {}
# relearning_side_effect_results = {}

model.cuda()
initial_test_loss = eval_callback(model, epoch=-1, forget_kwargs=relearn_forget_kwargs, maintain_kwargs=relearn_maintain_kwargs, inject_label=inject_label)

print(torch.cuda.memory_allocated() / 10**9, "GB")
print(train_tasks)
train_losses, test_losses = do_relearning(model, train_tasks, n_iters=n_relearn_iters, finetune_lora=True, lora_kwargs={'rank': 512, 'alpha': 32, 'dropout': 0.05, 'target_modules': 'all-linear'}, learning_kwargs={'lr': 2e-4, 'weight_decay': 0, 'use_cosine': True}, eval_callback_fn=eval_callback, forget_kwargs=relearn_forget_kwargs, maintain_kwargs=relearn_maintain_kwargs, inject_label=inject_label)

test_losses.insert(0, initial_test_loss)

for task_name, test_task in [("forget_sport", forget_sport_eval), ("maintain_sports", maintain_sports_eval)]:
    task_loss = 0
    task_accuracy = 0
    for i in range(n_eval_iters):
        task_loss += test_task.get_test_loss(model).item()
        task_accuracy += test_task.get_test_accuracy(model)
    relearning_regular_results[f"{task_name}_ce"] = task_loss / n_eval_iters
    relearning_regular_results[f"{task_name}_acc"] = task_accuracy / n_eval_iters
# model.cpu()

os.makedirs(f"{save_dir}/results", exist_ok=True)
with open(f"{save_dir}/results/relearning_results.pkl", "wb") as f:
    pickle.dump({"relearning_regular_results": relearning_regular_results, "relearning_train_losses": train_losses, "relearning_test_losses": test_losses}, f)